In [0]:
!rm -rf *

In [0]:
import cv2
import os
import glob
import shutil
import numpy as np

def exec_cmd(cmdstr):
  print(os.popen(cmdstr).read())
  
def download(url,fname=''):
  cmdstr = 'wget '+url+(' -O '+fname if fname else '')
  exec_cmd(cmdstr)

def convert(size, box):#box(xmin,xmax,ymin,ymax)
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def get_face_coords(class_name='',dir_name='.'):
  download('https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml','haarcascade_frontalface_default.xml')
  facedet = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

  img_names = glob.glob(dir_name+'/*.jpg')
  imgs = list(map(lambda imgf:cv2.imread(imgf,cv2.IMREAD_GRAYSCALE),img_names))
  rects = list(map(lambda img: facedet.detectMultiScale(img, scaleFactor = 1.2, minNeighbors = 5),imgs))
  

  img_data = dict(zip(img_names,imgs))
  rect_data = {}
  bbox_data = {}

  for i in range(len(img_names)):
    if len(rects[i])>0:
      rect_data[img_names[i]] = np.squeeze(np.array(rects[i])).tolist()
    else:
      print('No dets for ',img_names[i])

  for img_name in rect_data.keys():
    size = img_data[img_name].shape
    xmin,ymin,w,h = rect_data[img_name]
  
    box = xmin,xmin+w,ymin,ymin+h
    bbox_data[img_name] = convert(size,box)
  
  return class_name,bbox_data

def dw_urls(params,datadir='data'):
  catname,urls = params
  #exec_cmd('mkdir '+catname)
  exec_cmd('mkdir '+datadir)
  for i in range(len(urls)):
    download(urls[i],catname+'_'+str(i).rjust(2,'0')+'.jpg')
  exec_cmd('mv -v '+catname+'_* '+datadir+'/')

def save_class(item,classes):
  cname,data = item
  for imgf in data.keys():
    txtfname = '.'.join(imgf.split('.')[:-1])+'.txt'
    try:
      with open(txtfname,'w') as f:
        datastr = str(classes[cname])+' '+' '.join(list(map(str,data[imgf])) )
        f.write(datastr)
    except Exception as e:
      print(e)

def save_as_zip(dirname,zipname):
  try:
    shutil.make_archive(zipname, 'zip', dirname)
  except Exception as e:
    print(e)

In [0]:
urls={'modi': ['https://bsmedia.business-standard.com/_media/bs/img/article/2019-06/19/full/1560888256-2917.jpg',
'https://www.orfonline.org/wp-content/uploads/2019/05/modi-black-mr1.jpg',
'https://i0.wp.com/cdn.inc42.com/wp-content/uploads/2018/02/modi.jpg',
'https://yt3.ggpht.com/a/AGF-l7-nUWLbveRF3WkZDUeLBuJCt6JsZFJr3SCg3g=s900-mo-c-c0xffffffff-rj-k-no',
'http://images.newindianexpress.com/uploads/user/imagelibrary/2018/4/7/w600X300/Prime_Minister_Narendra_Modi_-_PTI.jpg',
'https://smedia2.intoday.in/btmt/images/stories/modi_cabinet_660_052919102513.jpg',
# 'https://cdn-live.theprint.in/wp-content/uploads/2019/06/Modi-696x391.jpeg',
# 'https://akm-img-a-in.tosshub.com/indiatoday/images/story/201904/PM-Modi-PTI_0.jpeg?pULs4CKClBaUt51Maal7ZJh60Ch7x6aK',
# 'https://www.informalnewz.com/wp-content/uploads/2019/06/Modi-3-770x433.jpg',
# 'https://images1.livehindustan.com/uploadimage/library/2019/06/14/16_9/16_9_1/russian_president_vladimir_putin_meets_with_indian_prime_minister_narendra_modi_on_sco_summit_1560492867.jpg',
'https://smedia2.intoday.in/btmt/images/stories/modi_ji_505_012519065412_062219060225.jpg',
'https://smedia2.intoday.in/btmt/images/stories/modi_660_022319034325_062319015054.jpg',
'https://static-news.moneycontrol.com/static-mcnews/2019/03/Narendra-Modi-2-770x433.jpg',
# 'https://img.etimg.com/thumb/height-480,width-640,msid-65839906,imgsize-221051/you-will-find-pm-modi-in-the-life-of-child-narendra.jpg',
'https://c.ndtvimg.com/2019-05/2ncrjufk_narendra-modi-pti_625x300_17_May_19.jpg'],

'rahul' : ['https://img-s-msn-com.akamaized.net/tenant/amp/entityid/AAD16r4.img',
'https://images1.livehindustan.com/uploadimage/library/2019/05/29/16_9/16_9_1/congress_president_rahul_gandhi_pti_file_photo__1559100841.jpg',
'https://images.in.com/uploads/2017/12/Rahul_Gandhi-e1514563507379-768x629.jpg',
'https://c.ndtvimg.com/2019-03/c9ruqonc_rahul-gandhi-pti_625x300_02_March_19.jpg',
'https://s3.ap-south-1.amazonaws.com/hansindia-bucket/h-upload/feeds/2019/05/04/171188-rahul-gandhi.jpg',
'https://static-news.moneycontrol.com/static-mcnews/2019/03/Rahul-Gandhi-6-770x433.jpg',
'https://smedia2.intoday.in/btmt/images/stories/rahul-gandhi_660_021119124629_022619113050_031919085650.jpg',
'https://iyc.in/wp-content/uploads/2018/12/rahul-gandhi-inspiration-iyc.jpg',
'https://cdn1.newsstate.com/images/2019/05/26/rahul-gandhi-3-24_5.jpg',
'http://ibgnews.com/wp-content/uploads/2018/09/Rahul-Gandhi.png']
}

In [38]:
datadir = 'data'
classes = dict(zip(urls.keys(),list(range(len(urls)))))

list(map(lambda u:dw_urls(u,datadir),urls.items()))
data = { cn:bbdata for cn,bbdata in list(map( lambda d: get_face_coords(d,'data') ,urls.keys()))}
list(map( lambda d:save_class(d,classes) ,data.items()))

with open('data/classes.txt','w') as f:
  f.write('\n'.join(classes))

save_as_zip('data','data')












renamed 'modi_00.jpg' -> 'data/modi_00.jpg'
renamed 'modi_01.jpg' -> 'data/modi_01.jpg'
renamed 'modi_02.jpg' -> 'data/modi_02.jpg'
renamed 'modi_03.jpg' -> 'data/modi_03.jpg'
renamed 'modi_04.jpg' -> 'data/modi_04.jpg'
renamed 'modi_05.jpg' -> 'data/modi_05.jpg'
renamed 'modi_06.jpg' -> 'data/modi_06.jpg'
renamed 'modi_07.jpg' -> 'data/modi_07.jpg'
renamed 'modi_08.jpg' -> 'data/modi_08.jpg'
renamed 'modi_09.jpg' -> 'data/modi_09.jpg'












renamed 'rahul_00.jpg' -> 'data/rahul_00.jpg'
renamed 'rahul_01.jpg' -> 'data/rahul_01.jpg'
renamed 'rahul_02.jpg' -> 'data/rahul_02.jpg'
renamed 'rahul_03.jpg' -> 'data/rahul_03.jpg'
renamed 'rahul_04.jpg' -> 'data/rahul_04.jpg'
renamed 'rahul_05.jpg' -> 'data/rahul_05.jpg'
renamed 'rahul_06.jpg' -> 'data/rahul_06.jpg'
renamed 'rahul_07.jpg' -> 'data/rahul_07.jpg'
renamed 'rahul_08.jpg' -> 'data/rahul_08.jpg'
renamed 'rahul_09.jpg' -> 'data/rahul_09.jpg'





In [28]:
data

{'modi': {}, 'rahul': {}}